In [1]:
import numpy as np
from numpy.linalg import norm
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import struct
import matplotlib.pyplot as plt

In [3]:
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio

In [6]:
with open('train-images.idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    print(data.shape)
    X_train = data.reshape((size, nrows*ncols))

with open('train-labels.idx1-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    y_train = data.reshape((size,))

with open('t10k-images.idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    X_test = data.reshape((size, nrows*ncols))

with open('t10k-labels.idx1-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    y_test = data.reshape((size,))

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(47040000,)
(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [9]:
# Run PCA on training data
pca = PCA()
pca.fit(X_train)

pcs = pca.components_[:16]

fig = sp.make_subplots(rows=4, cols=4, subplot_titles=[f'PC {i+1}' for i in range(16)], shared_xaxes=True, shared_yaxes=True)

for i in range(16):
    fig.add_trace(
        px.imshow(pcs[i].reshape(28, 28), color_continuous_scale='gray').data[0],
        row=i//4 + 1, col=i%4 + 1
    )

fig.update_layout(height=800, width=800, title_text="First 16 Principal Components", showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

pio.write_image(fig, "pca_components.pdf")

fig.show()

In [31]:
def compute_cumulative_energy(singular_values):
    cumulative_energy = np.cumsum(singular_values**2) / np.sum(singular_values**2)
    return cumulative_energy

def find_k_for_energy(cumulative_energy, threshold):
    k = np.argmax(cumulative_energy >= threshold) + 1 
    return k

def plot_reconstructed_images(pca, X, k):
    X_pca = pca.transform(X)
    X_reconstructed = pca.inverse_transform(X_pca)
    
    fig = sp.make_subplots(rows=2, cols=5, subplot_titles=[f'Image {i+1}' for i in range(10)], shared_xaxes=True, shared_yaxes=True)

    for i in range(10):
        row = i//5 + 1
        col = i%5 + 1
        fig.add_trace(
            px.imshow(X_reconstructed[i].reshape(28, 28), color_continuous_scale='viridis').data[0],
            row = row, col = col
        )

    fig.update_layout(height=400, width=1000, title_text=f'Reconstructed Images using {k} Principal Components', showlegend=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    pio.write_image(fig, "reconstructed_pca.pdf")

    fig.show()

# Perform SVD via PCA
pca = PCA()
pca.fit(X_train)

# Compute cumulative energy
cumulative_energy = compute_cumulative_energy(pca.singular_values_)
k = find_k_for_energy(cumulative_energy, threshold=0.85)
print(f'Number of principal components needed for 85% energy: {k}')

# Reconstruct images using k principal components
pca_k = PCA(n_components=k)
X_train_k = pca_k.fit_transform(X_train)
plot_reconstructed_images(pca_k, X_train[:10], k)

Number of principal components needed for 85% energy: 59


In [5]:
# Writing function to select subset of training and test data with certain digit

def select_digit_subset(X_train, y_train, X_test, y_test, digits):
    train_mask = np.isin(y_train, digits)
    test_mask = np.isin(y_test, digits)
    
    X_subtrain, y_subtrain = X_train[train_mask], y_train[train_mask]
    X_subtest, y_subtest = X_test[test_mask], y_test[test_mask]
    
    return X_subtrain, y_subtrain, X_subtest, y_subtest

In [35]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score

In [7]:
# Select subset of digits 1 and 8
X_subtrain, y_subtrain, X_subtest, y_subtest = select_digit_subset(X_train, y_train, X_test, y_test, digits=[1, 8])

# Project data onto k principal components
X_subtrain_pca = pca_k.transform(X_subtrain)
X_subtest_pca = pca_k.transform(X_subtest)

# Train Ridge Classifier
ridge_clf = RidgeClassifier()
cross_val_scores = cross_val_score(ridge_clf, X_subtrain_pca, y_subtrain, cv=5)
print(f'Cross-validation accuracy: {np.mean(cross_val_scores):.4f}')

# Train and test classifier
ridge_clf.fit(X_subtrain_pca, y_subtrain)
test_accuracy = ridge_clf.score(X_subtest_pca, y_subtest)
print(f'Test accuracy: {test_accuracy:.4f}')

Cross-validation accuracy: 0.9643
Test accuracy: 0.9801


In [8]:
# Select subset of digits 3 and 8
X_subtrain, y_subtrain, X_subtest, y_subtest = select_digit_subset(X_train, y_train, X_test, y_test, digits=[3, 8])

# Project data onto k principal components
X_subtrain_pca = pca_k.transform(X_subtrain)
X_subtest_pca = pca_k.transform(X_subtest)

# Train Ridge Classifier
ridge_clf = RidgeClassifier()
cross_val_scores = cross_val_score(ridge_clf, X_subtrain_pca, y_subtrain, cv=5)
print(f'Cross-validation accuracy: {np.mean(cross_val_scores):.4f}')

# Train and test classifier
ridge_clf.fit(X_subtrain_pca, y_subtrain)
test_accuracy = ridge_clf.score(X_subtest_pca, y_subtest)
print(f'Test accuracy: {test_accuracy:.4f}')

Cross-validation accuracy: 0.9587
Test accuracy: 0.9642


In [9]:
# Select subset of digits 2 and 7
X_subtrain, y_subtrain, X_subtest, y_subtest = select_digit_subset(X_train, y_train, X_test, y_test, digits=[2, 7])

# Project data onto k principal components
X_subtrain_pca = pca_k.transform(X_subtrain)
X_subtest_pca = pca_k.transform(X_subtest)

# Train Ridge Classifier
ridge_clf = RidgeClassifier()
cross_val_scores = cross_val_score(ridge_clf, X_subtrain_pca, y_subtrain, cv=5)
print(f'Cross-validation accuracy: {np.mean(cross_val_scores):.4f}')

# Train and test classifier
ridge_clf.fit(X_subtrain_pca, y_subtrain)
test_accuracy = ridge_clf.score(X_subtest_pca, y_subtest)
print(f'Test accuracy: {test_accuracy:.4f}')

Cross-validation accuracy: 0.9797
Test accuracy: 0.9748


In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
pca_k = PCA(n_components=k)
X_train_k = pca_k.fit_transform(X_train)
X_test_k = pca_k.transform(X_test)

# Train and evaluate Ridge Classifier
ridge_clf = RidgeClassifier()
cross_val_scores_ridge = cross_val_score(ridge_clf, X_train_k, y_train, cv=5)
print(f'Ridge Classifier cross-validation accuracy: {np.mean(cross_val_scores_ridge):.4f}')

ridge_clf.fit(X_train_k, y_train)
ridge_test_accuracy = ridge_clf.score(X_test_k, y_test)
print(f'Ridge Classifier test accuracy: {ridge_test_accuracy:.4f}')

# Train and evaluate KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
cross_val_scores_knn = cross_val_score(knn_clf, X_train_k, y_train, cv=5)
print(f'KNN Classifier cross-validation accuracy: {np.mean(cross_val_scores_knn):.4f}')

knn_clf.fit(X_train_k, y_train)
knn_test_accuracy = knn_clf.score(X_test_k, y_test)
print(f'KNN Classifier test accuracy: {knn_test_accuracy:.4f}')

Ridge Classifier cross-validation accuracy: 0.8436
Ridge Classifier test accuracy: 0.8563
KNN Classifier cross-validation accuracy: 0.9747
KNN Classifier test accuracy: 0.9755


In [33]:
from sklearn.svm import SVC

In [37]:
# SVM Classifier
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the SVM classifier
svm_clf = SVC(kernel="linear", random_state=42)

cross_val_scores = cross_val_score(svm_clf, X_train_scaled, y_train, cv=5)
print(f'Cross-validation accuracy: {np.mean(cross_val_scores):.4f}')

svm_clf.fit(X_train_scaled, y_train)

svm_test_accuracy = svm_clf.score(X_test_scaled, y_test)
print(f'SVM Classifier test accuracy: {svm_test_accuracy:.4f}')

Cross-validation accuracy: 0.9190
SVM Classifier test accuracy: 0.9293


In [38]:
import zipfile
import os

# Define the paths
notebook_name = "amath_482_hw3.ipynb"  
output_zip_name = "amath_482_hw3.zip" 

# Initialize a ZipFile object
with zipfile.ZipFile(output_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add the notebook
    zipf.write(notebook_name)
    
    for file_name in os.listdir():
        if file_name != notebook_name and not file_name.endswith('.zip'):
            zipf.write(file_name)

print(f"Notebook and files zipped into: {output_zip_name}")

Notebook and files zipped into: amath_482_hw3.zip
